# OpenSSL meets P4 AAAA via LoTW

Prototype for using OpenSSL to authenticate using ARRL Logbook of The World certificates.

User is first asked to run TQSL, go to the `Callsign Certificates` screen, select the desired callsign certificate, and click `Save a Callsign Certificate`. This saves all the crypto information (including the private parts) to a file. That filename is entered below as PKCS12_filename.

Everything below is implemented twice: once using the openssl command line tool, and again using Python's libraries `cryptography` and `OpenSSL` (aka pyopenssl).

In [16]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.serialization import pkcs12, Encoding
from cryptography import x509
import glob
import OpenSSL
import os
import re
import subprocess

os.chdir(os.path.expanduser('~/Desktop/AAAA_test'))
trusted_certs_filename = 'trusted/all_trusted.pem'
trusted_root_filename = 'trusted/arrlroot.pem'
trusted_prod_filename = 'trusted/arrlprod.pem'
PKCS12_filename = "KB5MU.p12"

process = subprocess.run(['ls', '-l', PKCS12_filename])

-rw-r--r-- 1 kb5mu kb5mu 6691 Jun 22 14:23 KB5MU.p12


Our first task is to find out the callsign associated with this certificate. The PKCS12 file format include "Subject" information that describes what the certificate applies to, in a standardized way. ITU X.520 https://www.itu.int/rec/T-REC-X.520-201910-I/en specifies the format, a Relative Distinguished Name. One of the ways this can be specified is with a Private Enterprise Number, which is a dot-separated sequence of numbers starting with `1.3.6.1.4.1` and followed by a number from the Enterprise Numbers list maintained by IANA at https://www.iana.org/assignments/enterprise-numbers/enterprise-numbers, plus some further sequence of numbers specified by that enterprise. The number for ARRL is `12348`, and they've apparently assigned `1.1` to mean amateur radio callsign.

With some pattern matching we can extract the callsign from the `-info` readable dump format of the PKCS12 file provided by the command-line openssl. Better, with the API we can get the callsign out semantically by checking for a match with the desired Object Identifier.

From here, we're assuming that the user doesn't have a password set on his private key, just to simplify the demonstration.

In [13]:
process = subprocess.run(['openssl', 'pkcs12', '-info', '-in', PKCS12_filename, '-password', 'pass:', '-nodes'],
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
p = re.compile(r'subject=/?1.3.6.1.4.1.12348.1.1 ?= ?(\w+)')    # two different string formats known; may be more!
m = p.search(process.stdout)
callsign = m.group(1)
callsign

'KB5MU'

In [33]:
with open(PKCS12_filename, "rb") as f:
    buffer = f.read()

p12 = pkcs12.load_pkcs12(buffer, None)
for name in p12.cert.certificate.subject.rdns:
    attributes = name.get_attributes_for_oid(x509.ObjectIdentifier('1.3.6.1.4.1.12348.1.1'))
    if attributes:
        callsign = attributes[0].value
        break
else:
    printf('Did not find a matching Object ID for the callsign')
    
callsign

'KB5MU'

Now we can grab our private key, public key, and certificates out of the PKCS12.

In [4]:
private_filename = callsign + '-private.pem'
public_filename = callsign + '-public.pem'
certs_filename = callsign + '-certs.pem'
mycert_filename = callsign + '-cert.pem'
pem_file_pattern = callsign + '-*.pem'

private_filename, public_filename, certs_filename, mycert_filename, pem_file_pattern

('KB5MU-private.pem',
 'KB5MU-public.pem',
 'KB5MU-certs.pem',
 'KB5MU-cert.pem',
 'KB5MU-*.pem')

In [5]:
process = subprocess.run(['openssl', 'pkcs12', '-in', PKCS12_filename, '-password', 'pass:', '-out', private_filename, '-nodes', '-nocerts'],
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

process = subprocess.run(['openssl', 'pkcs12', '-in', PKCS12_filename, '-password', 'pass:', '-out', certs_filename, '-nodes', '-nokeys'],
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

process = subprocess.run(['openssl', 'pkcs12', '-in', PKCS12_filename, '-password', 'pass:', '-out', mycert_filename, '-nodes', '-nokeys', '-clcerts'],
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
                        
with open(public_filename, "w") as f:
    process = subprocess.run(['openssl', 'x509', '-pubkey', '-in', mycert_filename, '-noout'],
                        stdout=f, stderr=subprocess.STDOUT, universal_newlines=True)

process = subprocess.run(['ls', '-l'] + glob.glob(pem_file_pattern))

-rw------- 1 kb5mu kb5mu 2360 Jun 23 23:48 KB5MU-cert.pem
-rw------- 1 kb5mu kb5mu 8079 Jun 23 23:48 KB5MU-certs.pem
-rw------- 1 kb5mu kb5mu 1587 Jun 23 23:48 KB5MU-private.pem
-rw-rw-r-- 1 kb5mu kb5mu  272 Jun 23 23:48 KB5MU-public.pem


In [34]:
# In the API, there are already accessors for everything we need:
[
p12.key,                             # my private key
p12.additional_certs,                # upstream certificates signing my certificate
p12.cert.certificate,                # my certificate
p12.cert.certificate.public_key(),   # my public key
]

 [<PKCS12Certificate(<Certificate(subject=<Name(C=US,ST=CT,L=Newington,O=American Radio Relay League,OU=Logbook of the World,CN=Logbook of the World Production CA,DC=arrl.org,1.2.840.113549.1.9.1=lotw@arrl.org)>, ...)>, friendly_name=None)>,
  <PKCS12Certificate(<Certificate(subject=<Name(C=US,ST=CT,L=Newington,O=American Radio Relay League,OU=Logbook of the World,CN=Logbook of the World Root CA,DC=arrl.org,1.2.840.113549.1.9.1=lotw@arrl.org)>, ...)>, friendly_name=None)>],
 <Certificate(subject=<Name(1.3.6.1.4.1.12348.1.1=KB5MU,CN=PAUL T WILLIAMSON,1.2.840.113549.1.9.1=paul@mustbeart.com)>, ...)>,

At this point, we (the ground station) would send our authentication request to the payload, including my certificate. This certificate contains and authenticates my public key, as we can confirm by examining it:

In [7]:
process = subprocess.run(['openssl', 'x509', '-in', mycert_filename, '-noout', '-text'])

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number: 509672 (0x7c6e8)
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: C = US, ST = CT, L = Newington, O = American Radio Relay League, OU = Logbook of the World, CN = Logbook of the World Production CA, DC = arrl.org, emailAddress = lotw@arrl.org
        Validity
            Not Before: Mar 12 12:48:30 2020 GMT
            Not After : Mar 12 12:48:30 2023 GMT
        Subject: 1.3.6.1.4.1.12348.1.1 = KB5MU, CN = PAUL T WILLIAMSON, emailAddress = paul@mustbeart.com
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                RSA Public-Key: (1024 bit)
                Modulus:
                    00:a3:37:b4:d8:22:d7:a7:ab:0c:18:78:36:65:37:
                    2f:a4:5b:a1:43:e1:e5:8a:92:4c:82:0a:d9:55:e7:
                    79:d2:e9:e9:92:5b:d2:10:64:17:75:28:74:89:20:
                    65:cc:dd:61:46:cd:52:c4:84:1b:31:b3:47:b4:f6:
                    c5:34:8e:08:

The payload will need to validate this certificate against the trusted LoTW root certificate and production certificate(s), which it already knows.

In [8]:
process = subprocess.run(['openssl', 'verify', '-CAfile', trusted_certs_filename, mycert_filename])

KB5MU-cert.pem: OK


Unfortunately, `cryptography` doesn't have a nice way to check a certificate. We'll have to use the older `OpenSSL` library for that.

In [9]:
mycert = OpenSSL.crypto.X509.from_cryptography(p12.cert.certificate)

with open(trusted_root_filename, "r") as f:
    c = f.read()
    arrlroot = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM,c)

with open(trusted_prod_filename, "r") as f:
    c = f.read()
    arrlprod = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM,c)

# Trust ARRL's root certificate
store = OpenSSL.crypto.X509Store()
store.add_cert(arrlroot)

# only add production certificate if it can be verified by root
ctx = OpenSSL.crypto.X509StoreContext(store, arrlprod)
ctx.verify_certificate()
try:
    ctx.verify_certificate()
except:
    print('Problem with production certificate')
else:
    print('Production certificate verified')
    store.add_cert(arrlprod)

# now try to verify the offered certificate
ctx = OpenSSL.crypto.X509StoreContext(store, mycert)
try:
    ctx.verify_certificate()
except:
    print('User certificate not accepted')
else:
    print('User certificate OK')

Production certificate verified
User certificate OK


The payload will now want to extract our public key from the certificate, so it can retain the public key for checking our signatures. In fact, it will probably want to convert it to binary to minimize storage costs.

In [10]:
payload_saved_public_filename = "saved_public.pem"
payload_saved_public_binary_filename = "saved_public.der"

with open(payload_saved_public_filename, "w") as f:
    process = subprocess.run(['openssl', 'x509', '-pubkey', '-in', mycert_filename, '-noout'],
                        stdout=f, stderr=subprocess.STDOUT, universal_newlines=True)

process = subprocess.run(['ls', '-l', payload_saved_public_filename])

process = subprocess.run(['openssl', 'rsa', '-pubin', '-inform', 'pem', '-in', payload_saved_public_filename, '-out', payload_saved_public_binary_filename, '-outform', 'der'],
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

process = subprocess.run(['ls', '-l', payload_saved_public_binary_filename])


-rw-rw-r-- 1 kb5mu kb5mu 272 Jun 23 23:48 saved_public.pem
-rw------- 1 kb5mu kb5mu 162 Jun 23 23:48 saved_public.der


In [11]:
payload_saved_public_key = p12.cert.certificate.public_key()
payload_saved_public_key

We are now in a position to sign things (using our private key) and the payload is in a position to verify that signature (using our public key, which it will have had to retain). Like so:

In [20]:
high_value_message = "My name is Ozymandias, King of Kings"
message_filename = 'message.dat'
signature_filename = 'signature.sha256'

# create a file to sign
with open(message_filename, "w") as f:
    f.write(high_value_message)

# on the ground, sign the file with our private key
process = subprocess.run(['openssl', 'dgst', '-sha256', '-sign', private_filename, '-out', signature_filename, message_filename ],
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

process = subprocess.run(['ls', '-l', signature_filename])

# in the payload, verify the signature with our public key
process = subprocess.run(['openssl', 'dgst', '-sha256', '-keyform', 'der', '-verify', payload_saved_public_binary_filename, '-signature', signature_filename, message_filename])


-rw-rw-r-- 1 kb5mu kb5mu 128 Jun 23 23:55 signature.sha256
Verified OK


In [31]:
# for cryptography we need the message as bytes
hv_message = bytes(high_value_message, 'ascii')

# on the ground, sign the message with our private key
signature = p12.key.sign(hv_message,
                        padding.PKCS1v15(   # for legacy compatibility
#                        padding.PSS(       # recommended for new applications
#                        mgf=padding.MGF1(hashes.SHA256()),
#                        salt_length=padding.PSS.MAX_LENGTH
                        ),
                        hashes.SHA256())

signature.hex()

# in the payload, verify the signature with our public key
try:
    payload_saved_public_key.verify(signature, hv_message,
                        padding.PKCS1v15(   # for legacy compatibility
#                        padding.PSS(       # recommended for new applications
#                        mgf=padding.MGF1(hashes.SHA256()),
#                        salt_length=padding.PSS.MAX_LENGTH
                        ),
                        hashes.SHA256())
except:
    print("Signature rejected")
else:
    print("Signature OK")

Signature OK


# Ta Dah!

We've demonstrated that we can use the ARRL's LoTW public key infrastructure to securely authenticate our callsign identity and sign messages, and the payload can verify their authenticity without needing to know any of ARRL's secrets.

The security here is, of course, limited by how secure ARRL's authentication of licensed radio amateurs is. In the United States, ARRL sends a postcard to the applicant's FCC-registered mailing address. That's as secure as anything the Federal government uses for amateur radio licensees. That may not be saying a whole lot, but there's no point in us trying to be more secure than the licensing body. For amateurs outside the United States, ARRL requires them to email images of proof of their license status. Presumably ARRL looks at these documents and perhaps cross-checks them with available databases when possible. This is roughly the same amount of scrutiny the FCC would give to reciprocal license applicants, so I think we can be reasonably assured that this is sufficient.

The other limit on security is the individual amateur's ability and incentive to keep their private key a secret. Logbook of The World also relies on this. If a private key is known to be compromised, certificates can be revoked and reissued. I don't know how often that happens, or how difficult the procedure might be. If our payload is to automatically take advantage of this mechanism, it would have to perform some transactions on the Internet for each certificate verification. That wouldn't necessarily have to happen in real time.

A payload with full-featured Authentication and Authorization needs to have the capability to maintain a block list of stations not permitted to use the system. The need to handle revoked certificates cleanly points to a requirement that the block list be able to distinguish between permanent blocks on a callsign (say, for bad behavior) and blocks due to compromised private keys. Probably it would be enough if each blocked callsign also stored a date. Any certificates for that callsign older than the stored date would be rejected. Permanant blocks would just have a date in the very far future.
